# N-PUZZLE GREEDY LAYER SOLUTION

In [518]:
import heapq
import json
from collections import namedtuple
import random
from tqdm.auto import tqdm
import numpy as np

In [519]:
PUZZLE_DIM = 50 # Change the dimension to test
RANDOMIZE_STEPS = 100_000 * (PUZZLE_DIM//3)
FINAL_STATE = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))

action = namedtuple('Action', ['pos1', 'pos2'])

## Helper functions

In [520]:
def available_actions(state: np.ndarray, layer=0, preferencial_dir=None) -> list["Action"]:
    '''Returns the available actions based on the current layer that we are solving, and an optional preferencial direction'''
    x, y = [int(_[0]) for _ in np.where(state == 0)]  
    actions = []  
    
    # move definition
    if x > layer :  # up
        actions.append(action((x, y), (x - 1, y)))
    if y < PUZZLE_DIM - 1:  # right
        actions.append(action((x, y), (x, y + 1)))
    if y > 0:  # left
        actions.append(action((x, y), (x, y - 1)))
    if x < PUZZLE_DIM - 1:  # down
        actions.append(action((x, y), (x + 1, y)))
    
    # preferencial directions
    direction_order = {
        "up": ["up", "right", "left", "down"],
        "down": ["down", "right", "left", "up"],
        "right": ["right", "down", "up", "left"],
        "left": ["left", "down", "up", "right"],
        None: ["left","up","right", "down"], 
    }
    
    # Map actions and strings if they are present
    action_map = {
        "up": action((x, y), (x - 1, y)) if x > layer else None,
        "right": action((x, y), (x, y + 1)) if y < PUZZLE_DIM - 1 else None,
        "left": action((x, y), (x, y - 1)) if y > 0 else None,
        "down": action((x, y), (x + 1, y)) if x < PUZZLE_DIM - 1 else None,
    }
    
    # Ordina le azioni disponibili secondo il preferencial_dir
    sorted_actions = [
        action_map[dir] for dir in direction_order[preferencial_dir] if action_map[dir] is not None
    ]
    
    return sorted_actions

def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    ''' Change the current state applying one action'''
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

def is_final_sol(state: np.ndarray) -> bool:
    '''Check the final state'''
    return np.array_equal(state, FINAL_STATE)

## Random Initialization

Reproducible Initialization

In [521]:
random.seed(RANDOMIZE_STEPS)

curr_state = FINAL_STATE
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    curr_state = do_action(curr_state, random.choice(available_actions(curr_state)))
    
print("Initial state: ")    
curr_state

Randomizing: 100%|██████████| 1600000/1600000 [00:37<00:00, 43235.28it/s]

Initial state: 


array([[ 165,  110,  671, ...,  147,  699,  532],
       [ 464,  558,  453, ...,  450,  596,  148],
       [ 164,  215,  557, ...,  238,  893, 1097],
       ...,
       [1073, 2421, 2364, ..., 1198, 2299, 1991],
       [2123, 1755, 1915, ..., 2147, 1545, 1749],
       [2251, 2463, 2101, ..., 2133, 2441, 2176]])

## Distance Functions

In [522]:
def distance_between(x1,y1,x2,y2):
    '''Return the distance between two points'''
    return  np.abs(x1 - x2) + np.abs(y1 - y2)

## Move Functions

In [523]:
def actions_from_zero_to_point(state:np.ndarray,x,y,layer):
    '''Moves the zero (empty space) near to a specific point'''
    path = []
    current = state
    n = current[x,y]
    x0, y0 = np.where(current==0)
    last_two_flag = False
    if layer == PUZZLE_DIM-2:
        last_two_flag=True
    while distance_between(x,y,x0,y0) > 1:
        for a in available_actions(current,layer,"right"):
            next = do_action(current,a)
            nx0, ny0 = np.where(next==0)
            if distance_between(x,y,nx0,ny0)< distance_between(x,y,x0,y0) and (current[nx0,ny0] > n or last_two_flag):
                current = next
                path.append(current)
                x0, y0 = nx0, ny0
                break
    return path, current

In [524]:
def put_zero_below(state:np.ndarray, x, y,layer, dir, couple=None):
    '''Moves the zero (empty space) below a specific point'''
    path = []
    current = state
    n = current[x,y]
    x0, y0 = np.where(current==0)
    
    if couple != None and current[x0,y0-1] == couple:
        a = action((x0, y0), (x0-1, y0)) #up
        current = do_action(current,a)
        path.append(current)
        x0, y0 = np.where(current==0)

    while distance_between(x+1,y,x0,y0) != 0:
        for a in available_actions(current,layer,dir):
            next = do_action(current,a)
            nx0, ny0 = np.where(next==0)
            if next[x,y]==n and distance_between(x+1,y,nx0,ny0)< distance_between(x+1,y,x0,y0):
                current = next
                path.append(current)
                x0, y0 = nx0, ny0
                break
    return path, current

In [525]:
def slide_row(state:np.ndarray,layer):
    '''Slides the current layer to leave space for the last number of the row'''
    current = state
    path = []
    x0, y0 = np.where(current==0)
    d_x, d_y = x0 -1 , PUZZLE_DIM-1 
    while distance_between(d_x,d_y,x0,y0) != 0:
        for a in available_actions(current,layer,"up"):
            next = do_action(current,a)
            nx0, ny0 = np.where(next==0)
            if distance_between(d_x,d_y,nx0,ny0)< distance_between(d_x,d_y,x0,y0):
                current = next
                path.append(current)
                x0, y0 = nx0, ny0
                break
    return path, current
    

In [526]:
def put_last_right(state:np.ndarray):
    '''Moves in the correct position the last number of a certain layer with fixed moves'''
    current = state
    path = []
    x,y = np.where(current==0)
    move_set= [
        action((x, y), (x + 1, y)), #down
        action((x + 1, y), (x + 1, y - 1)), #left
        action((x + 1, y - 1), (x, y - 1)) #up
    ]
    for act in move_set:
        current = do_action(current,act)
        path.append(current)
        
    return path, current

In [527]:
def slide_row_back(state:np.ndarray,layer):
    '''Slides the current layer back put again in the correct position the row'''
    current = state
    path = []
    x0, y0 = np.where(current==0)
    d_x, d_y = x0 + 1 , 0
    while distance_between(d_x,d_y,x0,y0) != 0:
        for a in available_actions(current,layer,"left"):
            next = do_action(current,a)
            nx0, ny0 = np.where(next==0)
            if distance_between(d_x,d_y,nx0,ny0)< distance_between(d_x,d_y,x0,y0):
                current = next
                path.append(current)
                x0, y0 = nx0, ny0
                break
    return path, current

In [528]:
def action_move_right_col(state:np.ndarray,x,y,layer,x_f,y_f):
    '''moves a number to the correct y position'''
    path = []
    current = state
    n = current[x,y]
    x0, y0 = np.where(current==0)
    prev_x0,prev_y0 = x0,y0
    while y != y_f:
        if y<y_f:
            # move n right
            while (y0,x0)!=(y+1,x):
                same_line = x0==x
                for a in available_actions(current,layer,"right"):
                    next = do_action(current,a)
                    nx0, ny0 = np.where(next==0)
                    if next[x,y]==n and np.abs(y-ny0) < 2 and np.abs(x-nx0) < 2 and (nx0,ny0)!=(prev_x0,prev_y0):
                        if ( ( same_line and np.abs(ny0-(y+1))<=np.abs(y0-(y+1)) ) or ( not same_line and distance_between(nx0,ny0,x,y+1)<distance_between(x0,y0,x,y+1) ) ):
                            prev_x0, prev_y0 = x0,y0
                            current = next
                            path.append(current)
                            x0, y0 = nx0, ny0
                            break
            left = action((x0, y0), (x0, y0 - 1))
            current = do_action(current,left)
            path.append(current)
            x,y = np.where(current==n)
            x0, y0 = np.where(current==0)
        else:
            # move n left
            while (y0,x0)!=(y-1,x):
                same_line = x0==x
                for a in available_actions(current,layer,"left"):
                    next = do_action(current,a)
                    nx0, ny0 = np.where(next==0)
                    if next[x,y]==n and np.abs(y-ny0) < 2 and np.abs(x-nx0) < 2 and (nx0,ny0)!=(prev_x0,prev_y0):
                        if ( ( same_line and np.abs(ny0-(y-1))<=np.abs(y0-(y-1)) ) or ( not same_line and distance_between(nx0,ny0,x,y-1)<distance_between(x0,y0,x,y-1) ) ):
                            prev_x0, prev_y0 = x0,y0
                            current = next
                            path.append(current)
                            x0, y0 = nx0, ny0
                            break
            right = action((x0, y0), (x0, y0 + 1))
            current = do_action(current,right)
            path.append(current)
            x,y = np.where(current==n)
            x0, y0 = np.where(current==0)
    return path, current

In [529]:
def action_move_right_row(state:np.ndarray,x,y,layer,x_f,y_f):
    '''moves a number to the correct x position'''
    path = []
    current = state
    n = current[x,y]
    x0, y0 = np.where(current==0)
    
    prev_x0,prev_y0 = x0,y0
    while x != x_f:
        if x<x_f:
            # move n down
            while (y0,x0)!=(y,x+1):
                same_col = y0==y
                for a in available_actions(current,layer, "down"):
                    next = do_action(current,a)
                    nx0, ny0 = np.where(next==0)
                    if next[x,y]==n and distance_between(nx0,ny0,x+1,y)<distance_between(x0,y0,x+1,y):
                        prev_x0, prev_y0 = x0,y0
                        current = next
                        path.append(current)
                        x0, y0 = nx0, ny0
                        break
            up = action((x0, y0), (x0 - 1, y0))
            current = do_action(current,up)
            path.append(current)
            x,y = np.where(current==n)
            x0, y0 = np.where(current==0)
        else:
            # move n up
            if ((x,y)==(x_f+1,y_f) and (x0,y0)!=(x_f,y_f) and n>1 and y0 < y and n != (layer*PUZZLE_DIM)+PUZZLE_DIM):
                moves, current = put_zero_below(current,x,y,layer,"right")
                path.extend(moves)
                x,y = np.where(current==n)
                x0, y0 = np.where(current==0)
            while (y0,x0)!=(y,x-1):
                same_col = y0==y
                for a in available_actions(current,layer, "up"):
                    next = do_action(current,a)
                    nx0, ny0 = np.where(next==0)
                    if next[x,y]==n and np.abs(y-ny0) < 2 and np.abs(x-nx0) < 2 and (nx0,ny0)!=(prev_x0,prev_y0) :
                        if ( ( same_col and np.abs(nx0-(x-1)) <= np.abs(x0-(x-1)) ) or ( not same_col and distance_between(nx0,ny0,x-1,y)<distance_between(x0,y0,x-1,y)  ) ):
                            prev_x0, prev_y0 = x0,y0
                            current = next
                            path.append(current)
                            x0, y0 = nx0, ny0
                            break
            down = action((x0, y0), (x0 + 1, y0))
            current = do_action(current,down)
            path.append(current)
            x,y = np.where(current==n)
            x0, y0 = np.where(current==0)
    return path, current

In [530]:
def lead_n_to_pos(state:np.ndarray,n:int,layer,x_f,y_f):
    '''Moves a number to a specific point, considering layers and preferencial directions constraint'''
    current = state
    path = []
    x , y = np.where(current==n)
    x0, y0 = np.where(current==0)
    
    # Move zero near to number to move
    if ((x,y)!=(x_f,y_f)) and (np.abs(x-x0)>1 or np.abs(y-y0)>1):
        steps, current = actions_from_zero_to_point(current,x,y,layer)
        path.extend(steps)
        x , y = np.where(current==n)
    # move the number to right col
    steps, current = action_move_right_col(current,x,y,layer,x_f,y_f)
    path.extend(steps)
    x , y = np.where(current==n)
    # move number to right row
    steps, current = action_move_right_row(current,x,y,layer,x_f,y_f)
    path.extend(steps)
    return path, current


In [531]:
def put_cuple_near(state):
    '''Puts a cuple of number of the same final col one next other solving the last two layers'''
    path=[]
    current = state
    x0,y0 = np.where(current==0)
    moves = [
        action((x0, y0), (x0-1, y0)), # up
        action((x0-1, y0), (x0-1, y0 +1)), # right
        action((x0-1, y0+1), (x0, y0+1)), # down
        action((x0, y0+1), (x0, y0 +2)) # right
    ]
    for a in moves:
        current = do_action(current,a)
        path.append(current)
    return path,current
    

In [532]:
def put_cuple_right(state):
    '''Puts a cuple of number of the same final col in the right position'''
    path=[]
    current = state
    x0,y0 = np.where(current==0)
    moves = [
        action((x0, y0), (x0-1, y0)), # up
        action((x0-1, y0), (x0-1, y0 -1)), # left
        action((x0-1, y0-1), (x0-1, y0-2)), # left
        action((x0-1, y0-2), (x0, y0 -2)), # down
        action((x0, y0-2), (x0, y0 -1)) # right
    ]
    for a in moves:
        current = do_action(current,a)
        path.append(current)
    return path,current

## Layer Solving Functions

In [533]:
def solve_layer(state, layer):
    '''Solve a layer between 1 and DIM_PUZZLE-2.'''
    current = state
    path = []
    path.append(current)
    # solve first n-1 number in the line
    for n in range((layer*PUZZLE_DIM)+1,(layer*PUZZLE_DIM)+PUZZLE_DIM+1):
        x,y = np.where(current==n)
        x_f,y_f = np.where(FINAL_STATE==n)
        x0 , y0 = np.where(current==0)
        if ((x,y)!=(x_f,y_f)):
            if y_f == PUZZLE_DIM-1 and ((x0,y0)!=(x_f,y_f) or (x,y)!=(x_f+1,y_f)):
                x_f +=1
            moves_done, current = lead_n_to_pos(current,n,layer,x_f,y_f)
            path.extend(moves_done)
        
    if(current[layer,PUZZLE_DIM-1]==FINAL_STATE[layer,PUZZLE_DIM-1]): 
        return path,current    
    # move zero below the first number of the layer to be able to slide the row
    x1, y1 = np.where(current==(layer*PUZZLE_DIM)+1)
    moves_done, current = put_zero_below(current,x1,y1,layer+1,"left")
    path.extend(moves_done)

    #slide the row in order to leave space for the last number
    moves_done, current = slide_row(current,layer)
    path.extend(moves_done)

    # put last number right and zero to the left
    moves_done, current = put_last_right(current)
    path.extend(moves_done)

    #slide the row in order to leave space for the last number
    moves_done, current = slide_row_back(current,layer)
    path.extend(moves_done)

    return path,current

In [534]:
def solve_last_two_layers(state:np.ndarray):
    ''' Solve the last two layer together solving column per column at couple'''
    path=[]
    current=state
    layer = PUZZLE_DIM-2

    for col in range(PUZZLE_DIM-1):
        n_up = PUZZLE_DIM*(PUZZLE_DIM-2)+col+1
        n_down = PUZZLE_DIM*(PUZZLE_DIM-1)+col+1

        x_up_f, y_up_f = np.where(FINAL_STATE==n_up)
        x_down_f, y_down_f = np.where(FINAL_STATE==n_down)
        #First lead n_up to right pos
        moves,current = lead_n_to_pos(current,n_up,layer,x_up_f, y_up_f)
        path.extend(moves)

        # If n_down can be placed we place it
        if current[PUZZLE_DIM-1,col] == 0 and current[PUZZLE_DIM-1,col+1] == n_down:
            moves,current = lead_n_to_pos(current,n_down,layer,x_down_f,y_down_f)
            path.extend(moves)
            continue

        # if n_down is in the right place we go on, otherwise we put it two columns away from the right position
        if current[PUZZLE_DIM-1,col] != n_down:
            moves,current = lead_n_to_pos(current,n_down,layer,x_down_f,y_down_f+2)
            path.extend(moves)
        else:
            continue

        # Put zero below n_up and move it down (zero up and then right)
        moves, current = put_zero_below(current,x_up_f, y_up_f,layer,"left",n_down)
        path.extend(moves)
        
        moves, current = put_cuple_near(current)
        path.extend(moves)

        moves, current = put_cuple_right(current)
        path.extend(moves)
    
    # Puts the last one in the correct position
    last_one = PUZZLE_DIM*(PUZZLE_DIM-2)+ PUZZLE_DIM   
    x_last, y_last = np.where(FINAL_STATE==last_one)

    moves,current = lead_n_to_pos(current,last_one,layer,x_last,y_last)
    path.extend(moves)
    return path,current


## Layer Solver Caller

In [535]:
path = []
init_state = curr_state
for layer in range(PUZZLE_DIM-2):
    print(f"Solving Layer N: {layer+1}")
    moves,init_state = solve_layer(init_state, layer)
    print(f"N moves layer {layer+1}: {len(moves)}")
    path.extend(moves)
    print(f"Layer {layer+1} Solution:")
    print(init_state)

print(f"Solving Last two Layers")
moves, result = solve_last_two_layers(init_state)
print(f"N moves last two layers: {len(moves)}")
path.extend(moves)

Solving Layer N: 1
N moves layer 1: 5220
Layer 1 Solution:
[[   1    2    3 ...   48   49   50]
 [   0  671  962 ...  596  165  532]
 [ 110  558  719 ...  238  893  148]
 ...
 [1073 2421 2364 ... 1198 2299 1991]
 [2123 1755 1915 ... 2147 1545 1749]
 [2251 2463 2101 ... 2133 2441 2176]]
Solving Layer N: 2
N moves layer 2: 5130
Layer 2 Solution:
[[   1    2    3 ...   48   49   50]
 [  51   52   53 ...   98   99  100]
 [   0  504 1001 ...  148  671  532]
 ...
 [1073 2421 2364 ... 1198 2299 1991]
 [2123 1755 1915 ... 2147 1545 1749]
 [2251 2463 2101 ... 2133 2441 2176]]
Solving Layer N: 3
N moves layer 3: 5178
Layer 3 Solution:
[[   1    2    3 ...   48   49   50]
 [  51   52   53 ...   98   99  100]
 [ 101  102  103 ...  148  149  150]
 ...
 [1073 2421 2364 ... 1198 2299 1991]
 [2123 1755 1915 ... 2147 1545 1749]
 [2251 2463 2101 ... 2133 2441 2176]]
Solving Layer N: 4
N moves layer 4: 4346
Layer 4 Solution:
[[   1    2    3 ...   48   49   50]
 [  51   52   53 ...   98   99  100]
 [ 101

In [536]:
print(f"Total Moves: {len(path)}")
print(f"Final Solution:")
print(result)

Total Moves: 225838
Final Solution:
[[   1    2    3 ...   48   49   50]
 [  51   52   53 ...   98   99  100]
 [ 101  102  103 ...  148  149  150]
 ...
 [2351 2352 2353 ... 2398 2399 2400]
 [2401 2402 2403 ... 2448 2449 2450]
 [2451 2452 2453 ... 2498 2499    0]]
